In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from lightgbm import LGBMRegressor, Dataset
from sklearn.metrics import mean_squared_error as mse
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
########################### Load Data
#################################################################################
print('Load Main Data')
train_df = pd.read_csv(r'/content/drive/MyDrive/AAIC/Assignment - 22 Self Case Study 1/sales_train_evaluation.csv')
prices_df = pd.read_csv(r'/content/drive/MyDrive/AAIC/Assignment - 22 Self Case Study 1/sell_prices.csv')
calendar_df = pd.read_csv(r'/content/drive/MyDrive/AAIC/Assignment - 22 Self Case Study 1/calendar.csv')

Load Main Data


In [5]:
## Found this code snippet in one of the discussion notebooks, to downcast the int and float data types
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
calendar_df = reduce_mem_usage(calendar_df)

prices_df = reduce_mem_usage(prices_df)

train_df = reduce_mem_usage(train_df)

# submission = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
# submission = reduce_mem_usage(submission)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Mem. usage decreased to 96.13 Mb (78.8% reduction)


In [37]:
def function_1(test):
    # We are creating new features required for the prediction for days from 1942 till 1969
    for day in range(1942,1942+28):
      test['d_' + str(day)] = np.int32(0)

    # We are transforming our Time Series Data so that we can apply supervised ml problem techniques
    data = pd.melt(test, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
              var_name='day', value_name='demand').dropna()  

    ## Merging all the files so that we have them in 1 place to make features
    data = data.merge(calendar_df, left_on='day', right_on='d')
    data = data.merge(prices_df,on=['store_id','item_id', 'wm_yr_wk'], how='left')

    ## Imputing Null in the sell price with the mean of the product id
    data['sell_price'].fillna(data.groupby('id')['sell_price'].transform('mean'), inplace=True)

    #Feature 1 :: we are removing  _ from ex d_101 to get the day number i.e 101
    data['day'] = data['day'].apply(lambda x: x.split('_')[1]).astype(np.int16)

    #since weekday's are represented as wday with numbers and d is a duplicate column.
    data.drop(['d','weekday'], axis=1, inplace=True)

    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    #Imputing Nan
    for feature in nan_features:
        data[feature].fillna('NA', inplace = True)


    ##loading the dict which contains the mappings of the Label Encoders   
    all_label_dicts = pickle.load(open("/content/drive/MyDrive/AAIC/Assignment - 22 Self Case Study 1/all_label_dicts.p", "rb"))

    for feature in list(all_label_dicts.keys()):
      feat_dict = all_label_dicts[feature]
      data[feature] = data[feature].map(feat_dict)

    data['lag_28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_56'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56))

    data['rolling_mean_7']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_7']    = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())

    data['rolling_mean_56']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(7).mean())
    data['rolling_std_56']    = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(7).std())

    ## Creating time based features
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.week
    data['day_of_date'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek

    data['days'] = data['day']
    data['day'] = data['day_of_date']

    ## Weekend feature
    def weekend(arg):
        if arg==5 or arg==6:
            return 1
        else:
            return 0
    data['isweekend'] = data['dayofweek'].apply(weekend)

    data.fillna(0,inplace=True)

    best_model = pickle.load(open("/content/drive/MyDrive/AAIC/Assignment - 22 Self Case Study 1/best_model.p", "rb"))

    features = ['days','day', 'wm_yr_wk', 'wday', 'month', 'year', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'sell_price', 'week',
       'isweekend','lag_28', 'lag_56', 'rolling_mean_7', 'rolling_std_7',
       'rolling_mean_56', 'rolling_std_56']



    # We are splitting the data for validation and test and then predicting it's value
    Val = data[(data['days']>1913) & (data['days']<1942)]
    pred_val_array = best_model.predict(Val[features])

    Test = data[data['days']>1941]
    pred_test_array = best_model.predict(Test[features])
    

    # We are then reshaping the predicted value
    pred_val_array = np.reshape(pred_val_array, (-1, 28),order = 'F')
    pred_test_array = np.reshape(pred_test_array, (-1, 28),order = 'F')

    cols = ['F'+str(i) for i in range(1,29)]

    vals = pd.concat([pd.DataFrame([test['id']], index=[0]),pd.DataFrame(pred_val_array, columns=cols)],axis=1).rename(columns={0:'ID'})
    vals['ID'] = vals['ID'].apply(lambda x: x.replace('evaluation','validation'))
    tst = pd.concat([pd.DataFrame([test['id']], index=[0]),pd.DataFrame(pred_test_array, columns=cols)],axis=1).rename(columns={0:'ID'})

    return vals, tst

In [41]:
test = train_df.sample(random_state=13).reset_index(drop=True)
test

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_139_WI_1_evaluation,HOBBIES_1_139,HOBBIES_1,HOBBIES,WI_1,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,1,2,0,0,0,0,1,0,0,0,0,0,1,0,2,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0


In [43]:
%%time
Val_op,Test_op = function_1(test)
print('Forecast sales from days 1914 till 1941 is:')
display(Val_op)
print('\nForecast sales from days 1942 till 1969 is:')
display(Test_op)

Forecast sales from days 1914 till 1941 is:


,ID,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_139_WI_1_validation,0.542993,0.449291,0.434987,0.591991,0.581451,0.841939,0.797673,0.611378,0.470493,0.512672,0.416342,0.464339,0.501517,0.382907,0.364379,0.31791,0.334058,0.311306,0.338624,0.438959,0.44208,0.459217,0.520918,0.444174,0.512672,0.662136,0.72648,0.756494



Forecast sales from days 1942 till 1969 is:


,ID,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_139_WI_1_evaluation,0.51513,0.384972,0.460292,0.409489,0.361414,0.418519,0.418519,0.350823,0.30973,0.358559,0.466423,0.47653,0.536566,0.50746,0.434537,0.393107,0.465901,0.328942,0.358458,0.521976,0.504365,0.428928,0.361095,0.314402,0.33785,0.467537,0.431728,0.426228


CPU times: user 2.23 s, sys: 24.7 ms, total: 2.26 s
Wall time: 2.14 s


In [74]:
def function_2(test,test_op):
    # We are creating new features required for the prediction for days from 1942 till 1969
    for day in range(1942,1942+28):
      test['d_' + str(day)] = np.int32(0)

    # We are transforming our Time Series Data so that we can apply supervised ml problem techniques
    data = pd.melt(test, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
              var_name='day', value_name='demand').dropna()  

    ## Merging all the files so that we have them in 1 place to make features
    data = data.merge(calendar_df, left_on='day', right_on='d')
    data = data.merge(prices_df,on=['store_id','item_id', 'wm_yr_wk'], how='left')

    ## Imputing Null in the sell price with the mean of the product id
    data['sell_price'].fillna(data.groupby('id')['sell_price'].transform('mean'), inplace=True)

    #Feature 1 :: we are removing  _ from ex d_101 to get the day number i.e 101
    data['day'] = data['day'].apply(lambda x: x.split('_')[1]).astype(np.int16)

    #since weekday's are represented as wday with numbers and d is a duplicate column.
    data.drop(['d','weekday'], axis=1, inplace=True)

    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    #Imputing Nan
    for feature in nan_features:
        data[feature].fillna('NA', inplace = True)


    ##loading the dict which contains the mappings of the Label Encoders   
    all_label_dicts = pickle.load(open("/content/drive/MyDrive/AAIC/Assignment - 22 Self Case Study 1/all_label_dicts.p", "rb"))

    for feature in list(all_label_dicts.keys()):
      feat_dict = all_label_dicts[feature]
      data[feature] = data[feature].map(feat_dict)

    data['lag_28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_56'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56))

    data['rolling_mean_7']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_7']    = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())

    data['rolling_mean_56']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(7).mean())
    data['rolling_std_56']    = data.groupby(['id'])['demand'].transform(lambda x: x.shift(56).rolling(7).std())

    ## Creating time based features
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.week
    data['day_of_date'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek

    data['days'] = data['day']
    data['day'] = data['day_of_date']

    ## Weekend feature
    def weekend(arg):
        if arg==5 or arg==6:
            return 1
        else:
            return 0
    data['isweekend'] = data['dayofweek'].apply(weekend)

    data.fillna(0,inplace=True)

    best_model = pickle.load(open("/content/drive/MyDrive/AAIC/Assignment - 22 Self Case Study 1/best_model.p", "rb"))

    features = ['days','day', 'wm_yr_wk', 'wday', 'month', 'year', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'sell_price', 'week',
       'isweekend','lag_28', 'lag_56', 'rolling_mean_7', 'rolling_std_7',
       'rolling_mean_56', 'rolling_std_56']



    # Rmse for Test
    Test = data[(data['days']>1941)]
    pred_test_array = best_model.predict(Test[features])

    # We are then reshaping for calculating the rmse value
    y_pred = np.reshape(pred_test_array, (-1, 28),order = 'F')

    rmse_test = mse(y_true=test_op, y_pred=y_pred, squared=False)
    
    return rmse_test

In [75]:
rmse = function_2(test.iloc[:,:-28],test.iloc[:,-28:])

In [77]:
print("The RMSE of the test set is",rmse)

The RMSE of the test set is 0.42466958334746324
